In [1]:
import os, random, itertools, math, torch
from torch.utils.data import DataLoader
from transformers import (
    AutoTokenizer, AutoModelForMaskedLM,
    get_cosine_schedule_with_warmup
)
from torch.optim import AdamW
from datasets import load_dataset
device = "cuda" if torch.cuda.is_available() else "cpu"

In [2]:
!huggingface-cli login --token "your_hf_token"

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: write).
The token `mdlm` has been saved to /root/.cache/huggingface/stored_tokens
Your token has been saved to /root/.cache/huggingface/token
Login successful.
The current active token is: `mdlm`


In [ ]:
model_id = "<username>/diffusionlm"
tokenizer = AutoTokenizer.from_pretrained(model_id)
SEP_ID, CLS_ID, MASK_ID = tokenizer.sep_token_id, tokenizer.cls_token_id, tokenizer.mask_token_id
model = AutoModelForMaskedLM.from_pretrained(model_id, device_map=device)
model.eval();

tokenizer_config.json:   0%|          | 0.00/1.30k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/3.16M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/6.41M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/615 [00:00<?, ?B/s]

In [34]:
# Single forward pass:
prompt = "User: आप कैसे हैं?" + tokenizer.sep_token + " Assistant:"
prompt_ids = tokenizer.encode(prompt, add_special_tokens=False)
ans_len = 6
ids = [CLS_ID] + prompt_ids + [SEP_ID] + [MASK_ID]*ans_len + [SEP_ID]
with torch.no_grad():
  outs = model(input_ids=torch.tensor([ids]).to(device)).logits
print(outs.shape)
out_ids = outs[0].argmax(dim=-1).tolist()
print(tokenizer.decode(out_ids))

torch.Size([1, 18, 197285])
. Assistant : आप कैसे हैं?. Assistant :. Assistant : आप सहायक हूँ!.


In [36]:
prompt = "User: भारत की राजधानी कौनसा है? " + tokenizer.sep_token + " Assistant:"
prompt_ids = tokenizer.encode(prompt, add_special_tokens=False)
ans_len = 3
outs = None
ids = [CLS_ID] + prompt_ids + [SEP_ID] + [MASK_ID]*ans_len + [SEP_ID]
for i in range(ans_len):
  if i % 4 == 0: # Optional: only run through the model every 4 (i.e. keep the top 4 each forwrd pass)
    with torch.no_grad():
      outs = model(input_ids=torch.tensor([ids]).to(device)).logits
  out_probs = torch.softmax(outs[0], dim=-1)
  mask_locs = (torch.tensor(ids) == MASK_ID).nonzero(as_tuple=True)[0]
  new_probs = torch.zeros_like(out_probs)
  new_probs[mask_locs] = out_probs[mask_locs]
  max_probs, max_locs = new_probs.max(dim=-1)
  max_loc = max_probs.argmax(dim=-1)
  print(max_loc, tokenizer.decode(new_probs[max_loc].argmax().item()))
  ids[max_loc] = new_probs[max_loc].argmax().item()
print(tokenizer.decode(ids))

tensor(14, device='cuda:0') Assistant
tensor(15, device='cuda:0') :
tensor(16, device='cuda:0') दिल्ली
[CLS] User : भारत की राजधानी कौनसा है? [SEP] Assistant : [SEP] Assistant : दिल्ली [SEP]


In [37]:
# Wrapping that in a function
def sample(q, ans_len=32):
  prompt = f"User: {q} " + tokenizer.sep_token + " Assistant:"
  prompt_ids = tokenizer.encode(prompt, add_special_tokens=False)
  ids = [CLS_ID] + prompt_ids + [SEP_ID] + [MASK_ID]*ans_len + [SEP_ID]
  for i in range(ans_len):
    with torch.no_grad():
      outs = model(input_ids=torch.tensor([ids]).to(device)).logits
    out_probs = torch.softmax(outs[0], dim=-1)
    mask_locs = (torch.tensor(ids) == MASK_ID).nonzero(as_tuple=True)[0]
    new_probs = torch.zeros_like(out_probs)
    new_probs[mask_locs] = out_probs[mask_locs]
    max_probs, max_locs = new_probs.max(dim=-1)
    max_loc = max_probs.argmax(dim=-1)
    ids[max_loc] = new_probs[max_loc].argmax().item()
  return tokenizer.decode(ids)

In [43]:
sample("आप कौन हैं", ans_len=12)

'[CLS] User : आप कौन हैं [SEP] Assistant : [SEP] Assistant : नमस्ते! मैं कैसे आपकी मदद कर सकता हूँ? [SEP]'

In [67]:
sample("आप कैसे हैं?", ans_len=8)

'[CLS] User : आप कैसे हैं? [SEP] Assistant : [SEP] Assistant : नमस्ते! मैं कौन हूँ? [SEP]'

## Gradio Interface

In [70]:
import os, random, itertools, math, torch
from torch.utils.data import DataLoader
from transformers import (
    AutoTokenizer, AutoModelForMaskedLM,
    get_cosine_schedule_with_warmup
)
from torch.optim import AdamW
import torch
import numpy as np
import gradio as gr
import torch.nn.functional as F
from transformers import AutoTokenizer, AutoModel
import time
import re
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

Using device: cuda


In [ ]:
!huggingface-cli login --token "your_hf_token"

In [ ]:
# Load model and tokenizer
model_id = "<username>/diffusionlm"

tokenizer = AutoTokenizer.from_pretrained(model_id)
SEP_ID, CLS_ID, MASK_ID = tokenizer.sep_token_id, tokenizer.cls_token_id, tokenizer.mask_token_id
model = AutoModelForMaskedLM.from_pretrained(model_id)#, device_map=device)
model.eval()

In [71]:
MASK_TOKEN = "[MASK]"
# MASK_ID = 126336  # The token ID of [MASK] in LLaDA

In [72]:
def parse_constraints(constraints_text):
    """Parse constraints in format: 'position:word, position:word, ...'"""
    constraints = {}
    if not constraints_text:
        return constraints
        
    parts = constraints_text.split(',')
    for part in parts:
        if ':' not in part:
            continue
        pos_str, word = part.split(':', 1)
        try:
            pos = int(pos_str.strip())
            word = word.strip()
            if word and pos >= 0:
                constraints[pos] = word
        except ValueError:
            continue
    
    return constraints

In [73]:
def format_chat_history(history):
    """
    Format chat history for the LLaDA model
    
    Args:
        history: List of [user_message, assistant_message] pairs
        
    Returns:
        Formatted conversation for the model
    """
    conversation = ""
    for user_msg, assistant_msg in history:
        conversation += f"User: {user_msg} {tokenizer.sep_token} "
        if assistant_msg:
            conversation += f"Assistant: {assistant_msg} {tokenizer.sep_token} "
    return conversation

In [74]:
def add_gumbel_noise(logits, temperature):
    '''
    The Gumbel max is a method for sampling categorical distributions.
    According to arXiv:2409.02908, for MDM, low-precision Gumbel Max improves perplexity score but reduces generation quality.
    Thus, we use float64.
    '''
    if temperature <= 0:
        return logits
        
    logits = logits.to(torch.float64)
    noise = torch.rand_like(logits, dtype=torch.float64)
    gumbel_noise = (- torch.log(noise)) ** temperature
    return logits.exp() / gumbel_noise

In [75]:
def get_num_transfer_tokens(mask_index, steps):
    '''
    In the reverse process, the interval [0, 1] is uniformly discretized into steps intervals.
    Furthermore, because LLaDA employs a linear noise schedule (as defined in Eq. (8)),
    the expected number of tokens transitioned at each step should be consistent.

    This function is designed to precompute the number of tokens that need to be transitioned at each step.
    '''
    mask_num = mask_index.sum(dim=1, keepdim=True)

    base = mask_num // steps
    remainder = mask_num % steps

    num_transfer_tokens = torch.zeros(mask_num.size(0), steps, device=mask_index.device, dtype=torch.int64) + base

    for i in range(mask_num.size(0)):
        num_transfer_tokens[i, :remainder[i]] += 1

    return num_transfer_tokens

In [76]:
def generate_response_with_visualization(model, tokenizer, device, history, gen_length=32, 
                                                   constraints=None, recompute_every=4):
    """
    Generate text with LLaDA model with visualization using the same sampling as in generate.py
    
    Args:
        messages: List of message dictionaries with 'role' and 'content'
        gen_length: Length of text to generate
        steps: Number of denoising steps
        constraints: Dictionary mapping positions to words
        temperature: Sampling temperature
        cfg_scale: Classifier-free guidance scale
        block_length: Block length for semi-autoregressive generation
        remasking: Remasking strategy ('low_confidence' or 'random')
        
    Returns:
        List of visualization states showing the progression and final text
    """
    if not history:
        return [], ""
    
    last_user_message = history[-1][0]
    conversation_history = format_chat_history(history[:-1])
    
    prompt = conversation_history + f"User: {last_user_message} {tokenizer.sep_token} Assistant:"
    prompt_ids = tokenizer.encode(prompt, add_special_tokens=False)
    
    ids = [CLS_ID] + prompt_ids + [SEP_ID] + [MASK_ID] * gen_length + [SEP_ID]
    
    visualization_states = []
    initial_state = [(tokenizer.mask_token, "#444444") for _ in range(gen_length)]
    visualization_states.append(initial_state)
    
    old_ids = ids.copy()
    outs = None
    
    for i in range(gen_length):
        if i % recompute_every == 0:
            with torch.no_grad():
                outs = model(input_ids=torch.tensor([ids]).to(device)).logits
        
        out_probs = torch.softmax(outs[0], dim=-1)
        mask_locs = (torch.tensor(ids) == MASK_ID).nonzero(as_tuple=True)[0]
        
        if len(mask_locs) == 0:
            break
            
        new_probs = torch.zeros_like(out_probs)
        new_probs[mask_locs] = out_probs[mask_locs]
        
        max_probs, max_locs = new_probs.max(dim=-1)
        max_loc = max_probs.argmax(dim=-1)
        
        old_ids = ids.copy()
        ids[max_loc] = new_probs[max_loc].argmax().item()
        
        current_state = []
        response_start_idx = len([CLS_ID] + prompt_ids + [SEP_ID])
        
        for j in range(gen_length):
            pos = response_start_idx + j
            
            if ids[pos] == MASK_ID:
                current_state.append((tokenizer.mask_token, "#444444"))
            elif old_ids[pos] == MASK_ID:
                token = tokenizer.decode([ids[pos]], skip_special_tokens=True)
                confidence = float(max_probs[max_loc].cpu())
                if confidence < 0.3:
                    color = "#FF6666"
                elif confidence < 0.7:
                    color = "#FFAA33"
                else:
                    color = "#66CC66"
                current_state.append((token, color))
            else:
                token = tokenizer.decode([ids[pos]], skip_special_tokens=True)
                current_state.append((token, "#6699CC"))
        
        visualization_states.append(current_state)
    
    response_start_idx = len([CLS_ID] + prompt_ids + [SEP_ID])
    response_end_idx = response_start_idx + gen_length
    response_ids = ids[response_start_idx:response_end_idx]
    response_ids = [token_id for token_id in response_ids if token_id != MASK_ID]
    final_text = tokenizer.decode(response_ids, skip_special_tokens=True)
    
    return visualization_states, final_text

In [77]:
def bot_response(history, gen_length, delay, recompute_every):
    if not history:
        return history, [], ""
        
    try:
        vis_states, response_text = generate_response_with_visualization(
            model, tokenizer, device, 
            history, 
            gen_length=gen_length,
            recompute_every=recompute_every
        )
        
        history[-1][1] = response_text
        
        yield history, vis_states[0], response_text
        
        for state in vis_states[1:]:
            time.sleep(delay)
            yield history, state, response_text
            
    except Exception as e:
        error_msg = f"Error: {str(e)}"
        print(error_msg)
        
        error_vis = [(error_msg, "red")]
        
        yield history, error_vis, error_msg

In [78]:
def create_settings():
    with gr.Accordion("Generation Settings", open=False):
        with gr.Row():
            gen_length = gr.Slider(
                minimum=8, maximum=64, value=32, step=4,
                label="Generation Length"
            )
            recompute_every = gr.Slider(
                minimum=1, maximum=8, value=4, step=1,
                label="Recompute Logits Every N Steps"
            )
        with gr.Row():
            visualization_delay = gr.Slider(
                minimum=0.0, maximum=1.0, value=0.1, step=0.1,
                label="Visualization Delay (seconds)"
            )
    
    return gen_length, recompute_every, visualization_delay

In [81]:
css = '''
.category-legend{display:none}
button{height: 60px}
'''
def create_chatbot_demo():
    with gr.Blocks(css=css) as demo:
        gr.Markdown("# Diffusion Language Model Demo")
        gr.Markdown("Interactive demo showing the denoising process of the diffusion language model")
        
        chat_history = gr.State([])
        
        with gr.Row():
            with gr.Column(scale=3):
                chatbot_ui = gr.Chatbot(label="Conversation", height=500)
                
                with gr.Group():
                    with gr.Row():
                        user_input = gr.Textbox(
                            label="Your Message", 
                            placeholder="Type your message here...",
                            show_label=False
                        )
                        send_btn = gr.Button("Send")
                        
            with gr.Column(scale=2):
                output_vis = gr.HighlightedText(
                    label="Denoising Process Visualization",
                    combine_adjacent=False,
                    show_legend=True,
                )
        
        with gr.Accordion("Generation Settings", open=False):
            with gr.Row():
                gen_length = gr.Slider(
                    minimum=8, maximum=64, value=32, step=4,
                    label="Generation Length"
                )
                recompute_every = gr.Slider(
                    minimum=1, maximum=8, value=4, step=1,
                    label="Recompute Logits Every N Steps"
                )
            with gr.Row():
                visualization_delay = gr.Slider(
                    minimum=0.0, maximum=1.0, value=0.1, step=0.1,
                    label="Visualization Delay (seconds)"
                )
        
        clear_btn = gr.Button("Clear Conversation")
        
        def add_message(history, message, response):
            history = history.copy()
            history.append([message, response])
            return history
            
        def user_message_submitted(message, history):
            if not message.strip():
                return history, history, ""
                
            history = add_message(history, message, None)
            history_for_display = history.copy()
            message_out = ""
            
            return history, history_for_display, message_out
            
        def bot_response(history, gen_length, delay, recompute_every):
            if not history:
                return history, [], ""
                
            try:
                vis_states, response_text = generate_response_with_visualization(
                    model, tokenizer, device, 
                    history, 
                    gen_length=gen_length,
                    recompute_every=recompute_every
                )
                
                history[-1][1] = response_text
                
                yield history, vis_states[0], response_text
                
                for state in vis_states[1:]:
                    time.sleep(delay)
                    yield history, state, response_text
                    
            except Exception as e:
                error_msg = f"Error: {str(e)}"
                print(error_msg)
                error_vis = [(error_msg, "red")]
                yield history, error_vis, error_msg
        
        def clear_conversation():
            return [], [], []
        
        clear_btn.click(
            fn=clear_conversation,
            inputs=[],
            outputs=[chat_history, chatbot_ui, output_vis]
        )
        
        msg_submit = user_input.submit(
            fn=user_message_submitted,
            inputs=[user_input, chat_history],
            outputs=[chat_history, chatbot_ui, user_input]
        )
        
        send_click = send_btn.click(
            fn=user_message_submitted,
            inputs=[user_input, chat_history],
            outputs=[chat_history, chatbot_ui, user_input]
        )
        
        msg_submit.then(
            fn=bot_response,
            inputs=[chat_history, gen_length, visualization_delay, recompute_every],
            outputs=[chatbot_ui, output_vis]
        )
        
        send_click.then(
            fn=bot_response,
            inputs=[chat_history, gen_length, visualization_delay, recompute_every],
            outputs=[chatbot_ui, output_vis]
        )
        
    return demo

In [84]:
demo = create_chatbot_demo()
demo.queue().launch(share=False) # To create a public link, set `share=True`

/tmp/ipykernel_36/1658756857.py:16: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot_ui = gr.Chatbot(label="Conversation", height=500)


* Running on local URL:  http://127.0.0.1:7863
* To create a public link, set `share=True` in `launch()`.
